In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from get_llm import get_llm
import os
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

### We need to define a State before creating a LangGraph application

In [ ]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


### Create an empty graph - this is like a container to which we will add nodes and edges

In [ ]:
graph_builder = StateGraph(State)

### Get an instance of an LLM - this can be a LangChain instance or plain OpenAI as illustrated in get_completion_client

In [ ]:
llm = get_llm()

In [ ]:
llm

### Define your agents - I am just defining one simple agent
### You need to define a function and a name for the agent

In [ ]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

### Create your graph by adding nodes and edges

In [ ]:
# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

### START and END nodes are defined by LangGraph itself and can be imported
### Define the edges as per your design

In [ ]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

### Once the graph is created, compile it

In [ ]:
graph = graph_builder.compile()

### Optionally we can visualize the graph

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_png()))
# except Exception:
#     # This requires some extra dependencies and is optional
#     pass

### Execute the application using invoke - you need to provide the input and the initial state

In [ ]:
while True:
    prompt = input("Enter prompt: ")
    if prompt == "quit":
        break
    results = graph.invoke({"messages": prompt})
    print(results["messages"][-1])

In [ ]:
print(results["messages"][-1].content)

In [ ]:
print(results["messages"][-1].usage_metadata)